# Introduction

This notebook presents how to construct the Ultimate Forward Rate term structures, as prescribed by the Dutch National Bank (DNB), using the open source financial library `QuantLib`. It demonstrates the procedure to build term structures and to compute basic risk metrics. Also, a brief summary of the methodology is given to better understand the implementation.


# FTK Curve Construction

In the first step, before constructing the Ultimate Forward Rate curve, we need to bootstrap a term structure using the market input. 

The DNB-FTK methodology gives clear instructions regarding the market instruments that should be used, as well as the interpolation algorithm to be applied between the nodal points. The eligible market rates are CMPL Euribor 6m swaps (BID prices) for tenors 1 - 10, 12, 15, 20, 25, 30, 40 and 50 years.

The required interpolation method assumes flat annually compounded forward rate, which can be expressed as:

\begin{equation*}
(1+z(t,T_{i}))^{T_{i}-t} \times (1 + f(t,T_{i},T_{i+1}))^{(T_{i+1} - T_{i})} = (1+z(t,T_{i+1}))^{T_{i+1}-t},
\end{equation*}

where, $z(t,T_{i})$ is the zero (annually compounded) rate with maturity at time $T_{i}$ and $f(t,T_{i},T_{i+1})$ is the annually compounded forward rate between $T_{i}$ and $T_{i+1}$. 

The forward rate can be equivalently expressed in terms of discount factors as,

\begin{equation*}
(1 + f(t,T_{i},T_{i+1}))^{(T_{i+1} - T_{i})} = \left [ \frac{P(t,T_{i})}{P(t,T_{i+1})} \right ],
\end{equation*}

because each discount factor can be presented as a function of annually compounded zero rate: 

\begin{equation*}
P(t,T_{i}) = (1+z(t,T_{i}))^{-(T_{i}-t)}.
\end{equation*}

The interpolation used in the FTK curve assumes that between nodal points $[T_{i}, T_{i+1}]$ the annually compounded forward rate is constant. This can be summarized as follows:

\begin{equation*}
P(t,T) = P(t,T_{i}) \times (1 + \underbrace{f(t,T_{i},T_{i+1})}_{\text{constant}})^{-(T - T_{i})}, 
\end{equation*}

where $T_{i} \leq T \leq T_{i+1}$.

Using the previous equation, we can relate the interpolated discount factor entirely to the neighbouring discount factors at nodes: 

\begin{equation*}
P(t,T) = P(t,T_{i}) \times \left[ \frac{P(t,T_{i})}{P(t,T_{i+1})}\right]^{\frac{T_{i} - T}{T_{i+1} - T_{i}}},
\end{equation*}

where $T_{i} \leq T \leq T_{i+1}$.

An important consideration which may simplify the implementation is an observation that the above interpolation scheme is equivalent to interpolating linearly on log-discounts.

That scheme can be summarized as follows:

\begin{equation*}
\log P(t,T) = \log P(t,T_{i})  + \frac{T - T_{i}}{T_{i+1} - T_{i}} [\log P(t, T_{i+1}) - \log P(t, T_{i})],
\end{equation*}

where $T_{i} \leq T \leq T_{i+1}$.

After some reorganization of the equation we obtain,

\begin{equation*}
\log P(t,T) = \frac{T_{i+1} - T}{T_{i+1} - T_{i}} \log P(t,T_{i})  + \frac{T - T_{i}}{T_{i+1} - T_{i}} \log P(t, T_{i+1}).
\end{equation*}


Using basic properties of a logarithmic function we get,

\begin{equation*}
\log P(t,T) = \log \left [ P(t,T_{i})^{\frac{T_{i+1} - T}{T_{i+1} - T_{i}}} \right ] + \log \left [ P(t, T_{i+1})^{ \frac{T - T_{i}}{T_{i+1} - T_{i}}}\right ].
\end{equation*}

Taking the exponent on both sides of the equation and reorganizing according to the properties of an exponential function,

\begin{equation*}
P(t,T) = \exp \left \{ \log \left [ P(t,T_{i})^{\frac{T_{i+1} - T}{T_{i+1} - T_{i}}} \right ] \right \} \exp \left \{ \log \left [ P(t, T_{i+1})^{ \frac{T - T_{i}}{T_{i+1} - T_{i}}}\right ] \right \}.
\end{equation*}

Finally, we arrive at,

\begin{equation*}
P(t,T) = P(t,T_{i})^{\frac{T_{i+1} - T}{T_{i+1} - T_{i}}} \times P(t, T_{i+1})^{ \frac{T - T_{i}}{T_{i+1} - T_{i}}} = P(t,T_{i}) \times \left[ \frac{P(t,T_{i})}{P(t,T_{i+1})}\right]^{\frac{T_{i} - T}{T_{i+1} - T_{i}}},
\end{equation*}

which proves the equivalence of the flat compounded forward interpolation with linear log-discount interpolation.

In [ ]:
import QuantLib as ql

import csv
from typing import List, Tuple


def read_swap_quotes(date: ql.Date) -> List[Tuple[ql.Period, ql.RelinkableQuoteHandle]]:
    date_as_int = date.year() * 10000 + date.month() * 100 + date.dayOfMonth()
    file_path = 'data/swap_rates_' + str(date_as_int) + '.csv'
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        rates = [(ql.PeriodParser.parse(str(r[0])), float(r[1])) for r in reader]
        return [(q[0], ql.RelinkableQuoteHandle(ql.SimpleQuote(q[1]))) for q in rates]


def read_liabilities() -> List[Tuple[ql.Date, float]]:
    file_path = 'data/liabilities.csv'
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        return [(ql.Date(str(r[0]), '%Y%m%d'), float(r[1])) for r in reader]
    

def read_dnb_ufr_zero_rates(date: ql.Date):
    date_as_int = date.year() * 10000 + date.month() * 100 + date.dayOfMonth()
    file_path = 'data/dnb_ufr_zero_rates_' + str(date_as_int) + '.csv'
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        return [(ql.PeriodParser().parse(str(r[0])), float(r[1])) for r in reader]
    

We need to introduce a number of helper functions, e.g. to retrieve market data or other inputs required for this exercise, just like the functions defined in the cell above. 

Note that `read_swap_quotes` reads a list of tuples of type `(ql.Period, ql.RelinkableQuoteHandle)`. 
Storing any market data information in `ql.RelinkableQuoteHandle` gives better control over the data, e.g. for the purposes of stress-testing or computing numerical sensitivities. 

# UFR Curve(s) Construction


At the moment of writing this note (beginning 2021) three different modifications of the UFR model (2015, 2021, 2024) were in use. These correspond to the year when the model came, or will come, to usage in the regulatory reporting. We will briefly go through all of them, for completeness. 

## Ultimate forward rate

According to the UFR methodology, the Ultimate Forward Rate is equal to 120 months' average of the annually compounded forwards between 20 and 21 years to maturity (for the 2015 version of the model), or between 30 and 31 years to maturity (for the 2024 version of the model), recorded at each month's end, i.e.

\begin{equation*}
UFR(t) = \frac{1}{120} \sum_{m \in M(t)}{f(m, T_{\text{FSP}}, T_{\text{FSP + 1}})},
\end{equation*}

where FSP is the first smoothing point, which falls at the 20 or 30 years tenor, for 2015 and 2025 models respectively. $M(t)$ is a set of last business days in a given month.

Recall that the relation between the continuous and annual rates:

\begin{equation*}
z_c(t,T) = \log [1 + z(t,T)],
\end{equation*}

\begin{equation*}
UFR_c(t) = \log [1 + UFR(t)].
\end{equation*}

The final UFR rate is rounded to the third decimal point.
On the basis of the earlier bootstrapped market/FTK curve, the UFR term structure is extrapolated past the FSP. 

## Last liquid forward rate - 2015 version

Next to the UFR rate, another smoothing component is the last liquid forward rate (LLFR).
The 2015 version of the model uses the following formula to obtain the LLFR:

\begin{equation*}
LLFR^{2015}(t) = \omega_{2015} \left ( f_{c}(t,T_{20},T_{25}) + \frac{1}{2}f_{c}(t,T_{20},T_{30})+\frac{1}{4}f_{c}(t,T_{20},T_{40})+\frac{1}{8}f_{c}(t,T_{20},T_{50}) \right ),
\end{equation*}

where $\omega_{2015} = \frac{8}{15}$.

## Last liquid forward rate - 2024 version

The 2024 version defines the LLFR as:

\begin{equation*}
LLFR^{2024}(t) = \omega_{2024} \left ( \frac{2}{3}f_{c}(t,T_{30},T_{40})+\frac{1}{3}f_{c}(t,T_{30},T_{50}) \right ),
\end{equation*}

where $\omega_{2024} = 1$.

On the top of that, the regulator requires to calculate the $LLFR^{2024}$ as the average of $LLFR^{2024}(t)$ computed on the last five business days of a given month.

Note here that $f_{c}(t,T_{N},T_{N+1})$ is a continuously compounded forward rate for period $[T_{N},T_{N+1}]$. 

## Term structure extrapolation

The extrapolation of the forward rate beyond FSP is equal to:

\begin{equation*}
f_{c}(t,T_{\text{FSP}},T_{\text{FSP}+h}) = UFR_{c}(t)+[LLFR(t)-UFR_{c}(t)]B(h),
\end{equation*}

where $B(h)=\frac{1-\exp (-\alpha h )}{\alpha h}$, with $\alpha^{2015} = 0.1$ and $\alpha^{2024} = 0.02$.

Finally, we obtain the extrapolated continuous zero rate past FSP,

\begin{equation*}
z_{c}(t,T_{\text{FSP}+h}) = \frac{(T_{\text{FSP}} - t) z_{c}(t, T_{\text{FSP}}) + (T_{\text{FSP} + h} - T_{\text{FSP}}) f_{c}(t,T_{\text{FSP}},T_{\text{FSP}+h})}{T_{\text{FSP} + h} - t}.
\end{equation*}

At last, we need to transform the continuous rates into annualized ones,

\begin{equation*}
z(t,T_{\text{FSP}+h}) = \exp [z_{c}(t,T_{\text{FSP}+h})] - 1.
\end{equation*}

These rates are being published on the DNB website with precision to the 5th decimal point.

In [ ]:
def calculate_last_liquid_forward(
        crv: ql.YieldTermStructureHandle, 
        fsp: ql.Period,
        weights: List[Tuple[ql.Period, float]],
        omega: float) -> float:
    d_counter = crv.dayCounter()
    reference_date = crv.referenceDate()
    fsp_date = reference_date + fsp
    llfr = 0.0
    for tenor, weight in weights:
        end_date = reference_date + tenor
        llfr += weight * crv.forwardRate(fsp_date, end_date, d_counter, ql.Continuous).rate()
    return llfr * omega


Note that `calculate_last_liquid_forward` implements the LLFR formula described above. Depending on the provided input for `weights`, `omega` and `fsp` the result will correspond to either 2015 or 2025 model.

In [ ]:
# SWAP INDEX CONVENTIONS

SETTLEMENT_DAYS = 2
BUSINESS_CONVENTION = ql.Unadjusted
DAY_COUNT = ql.SimpleDayCounter()
CALENDAR = ql.NullCalendar()
CCY = ql.EURCurrency()
FXD_FREQUENCY = ql.Annual
FLT_TENOR = ql.Period(6, ql.Months)


# FTK CURVE CONSTRUCTION FUNCTION

def build_ftk_curve(valuation_date: ql.Date, quote_handles: List[ql.QuoteHandle]) -> ql.YieldTermStructure:
    idx = ql.IborIndex("FTK_IDX", FLT_TENOR, SETTLEMENT_DAYS, CCY, CALENDAR, BUSINESS_CONVENTION, False, DAY_COUNT)
    settlement = CALENDAR.advance(today, SETTLEMENT_DAYS, ql.Days)
    instruments = [ql.SwapRateHelper(q, t, CALENDAR, FXD_FREQUENCY, BUSINESS_CONVENTION, DAY_COUNT, idx) 
                   for t, q in quote_handles]
    crv = ql.PiecewiseLogLinearDiscount(settlement, instruments, DAY_COUNT)
    crv.enableExtrapolation()
    return crv


`build_ftk_curve` constructs the market term structure according to the guidelines of the DNB. As mentioned in the previous section, in order to replicate the DNB market curve (which is the basis for further constructing the UFR curve) we need to use linear interpolation on the logarithms of the discount factors. That can be achieved with `ql.PiecewiseLogLinearDiscount` function.

The interest rate swap instruments (created via `ql.SwapRateHelper`) that are used to bootstrap the curve follow simplified market conventions - with no business date adjustments and day count fractions expressed as rational numbers.

The example below illustrates what the simple day counter does.

In [ ]:
# SIMPLE DAY COUNTER EXAMPLE:

d1 = ql.Date(1, ql.January, 2020)
d2 = ql.Date(1, ql.July, 2020)
d3 = ql.Date(1, ql.January, 2021)

print(f'Year fraction between {d1} and {d2} equals {DAY_COUNT.yearFraction(d1, d2)}')
print(f'Year fraction between {d2} and {d3} equals {DAY_COUNT.yearFraction(d2, d3)}')
print(f'Year fraction between {d1} and {d3} equals {DAY_COUNT.yearFraction(d1, d3)}')

In [ ]:
# UFR 2015 CONVENTIONS

FIRST_SMOOTHING_POINT_2015 = ql.Period(20, ql.Years)
ALPHA_2015 = 0.1;

OMEGA_2015 = 8.0 / 15.0
WEIGHTS_2015 = ((ql.Period(25, ql.Years), 1.0), 
                (ql.Period(30, ql.Years), 0.5), 
                (ql.Period(40, ql.Years), 0.25), 
                (ql.Period(50, ql.Years), 0.125))

ufr_2015_compounded = ql.InterestRate(0.018, DAY_COUNT, ql.Compounded, ql.Annual)
ufr_2015_continuous = ufr_2015_compounded.equivalentRate(ql.Continuous, ql.NoFrequency, 1.0).rate()
ufr_2015_handle = ql.QuoteHandle(ql.SimpleQuote(ufr_2015_continuous))

In [ ]:
# UFR 2024 CONVENTIONS

FIRST_SMOOTHING_POINT_2024 = ql.Period(30, ql.Years)
ALPHA_2024 = 0.02;

OMEGA_2024 = 1.0
WEIGHTS_2024 = ((ql.Period(40, ql.Years), 2.0 / 3.0), (ql.Period(50, ql.Years), 1.0 / 3.0))

ufr_2024_compounded = ql.InterestRate(0.016, DAY_COUNT, ql.Compounded, ql.Annual)
ufr_2024_continuous = ufr_2024_compounded.equivalentRate(ql.Continuous, ql.NoFrequency, 1.0).rate()
ufr_2024_handle = ql.QuoteHandle(ql.SimpleQuote(ufr_2024_continuous))

The above cells define the parameters for the UFR curve for both 2015 and 2024 definitions of the model. 

Note that, in line with what was mentioned in the section describing the methodology, the UFR rate is first calculated as an average of 120 historic, end of month, annually compounded forward rates. Those are stored as `ql.InterestRate` objects, converted to their continuous equivalents and later passed to the curve constructor as `ql.QuoteHandle`.

We do not calculate the average here for the sake of brevity, only assume the rates upfront. Those are consistent with the rates observed by end January 2021.

In [ ]:
# VALUATION DATE

today = CALENDAR.adjust(ql.Date(29, ql.January, 2021))
ql.Settings.instance().evaluationDate = today

# READ SWAP RATES

swap_quotes = read_swap_quotes(today)

# RESERVE A PLACEHOLDER FOR FTK CURVE HANDLE

ftk_handle = ql.RelinkableYieldTermStructureHandle()
ftk_handle.linkTo(build_ftk_curve(today, swap_quotes))

In `QuantLib` the evaluation date is managed via a global setting, which needs to be set prior to any performed calculations.

Recall that `read_swap_quotes` retrieves the swap rates with the associated tenors into a list of tuples of type `(ql.Period, ql.RelinkableQuoteHandle)` and can be inspected in a straightforward way.

In [ ]:
for tenor, quote in swap_quotes:
    print(f'Tenor: {tenor} Rate: {quote.value()}')

Before proceeding to the next section, we need to take a moment to demonstrate how to define an observer in `QuantLib` and then link it to an observable.

Observer can be defined by passing a function (callable) which is executed once the observer is notified by the observable about a change. So, we expect `notify_observer` to be executed and then to print a message.

We defined an observable as `ql.RelinkableQuoteHandle()` and subscribed it to the observer. Note that `ql.RelinkableQuoteHandle()` is a `ql.Observable`.

In [ ]:
observable = ql.RelinkableQuoteHandle()

def notify_observer():
    print(f'Observer notified! The value is now {observable.value()}')
    

observer = ql.Observer(notify_observer)

# Subscribe to observer
observer.registerWith(observable)

In the final step we update the observable (by linking it to a new quote) just to spot that the observer was indeed notified.

In [ ]:
observable.linkTo(ql.SimpleQuote(0.0))
observable.linkTo(ql.SimpleQuote(0.01))
observable.linkTo(ql.SimpleQuote(20.01))

In [ ]:
# LAST LIQUID FORWARD RATE 2015 OBSERVER

def llfr_2015():
    return calculate_last_liquid_forward(ftk_handle, FIRST_SMOOTHING_POINT_2015, WEIGHTS_2015, OMEGA_2015)


llfr_2015_handle = ql.RelinkableQuoteHandle(ql.SimpleQuote(llfr_2015()))

def update_llfr_handle_2015():
    llfr_2015_handle.linkTo(ql.SimpleQuote(llfr_2015()))
    

llfr_2015_observer = ql.Observer(update_llfr_handle_2015)
for _, quote_handle in swap_quotes:
    llfr_2015_observer.registerWith(quote_handle)


The above presented reasoning is now applied to notify an "observer" about updates to the Last Liquid Forward Rate, which is defined here as a quote handle. That observer is the UFR curve.

The purpose of this setup is to ensure that any alteration of the market rates in `swap_quotes` will not only be immediately reflected on the `ftk_handle` (which by construction is an observer of those), but also on the LLFR. 

The end goal is to create a chain reaction in the UFR curve and update all its components that are passed as handles (`ftk_handle`, `llfr_2015_handle` or `ufr_2024_handle`) after changes in any of its constituents. 

In [ ]:
# LAST LIQUID FORWARD RATE 2024 OBSERVER

AVERAGE_POINTS = 5

# BUILD FTK CURVES FOR 4 PREVIOUS BUSINESS DAYS

previous_valuation_dates = [CALENDAR.advance(today, -i, ql.Days) for i in range(1, AVERAGE_POINTS)]
previous_ftk_curves_handles = [ql.YieldTermStructureHandle(build_ftk_curve(today, read_swap_quotes(dt))) 
                               for dt in previous_valuation_dates]
previous_ftk_curves_handles.append(ftk_handle)

def llfr_2024():
    return sum([calculate_last_liquid_forward(crv_hndl, FIRST_SMOOTHING_POINT_2024, WEIGHTS_2024, OMEGA_2024) 
                for crv_hndl in previous_ftk_curves_handles]) / AVERAGE_POINTS


llfr_2024_handle = ql.RelinkableQuoteHandle(ql.SimpleQuote(llfr_2024()))

def update_llfr_handle_2024():
    llfr_2024_handle.linkTo(ql.SimpleQuote(llfr_2024()))
    

llfr_2024_observer = ql.Observer(update_llfr_handle_2024)
for _, quote_handle in swap_quotes:
    llfr_2024_observer.registerWith(quote_handle)

Same procedure is applied for 2024 LLFR. The only relevant difference is that it consists of the LLFRs observed on the last five business days of the month.

A careful reader will notice that all curves are build with the same valuation date. `today` is still being passed instead of `dt`. The most "correct" approach would be to update `ql.Settings.instance().evaluationDate`. That would become a very cumbersome task in the context of this exercise. Secondly, given the fact that this setup assumes that every day in the calendar year is a business day, the resulting miscalculation is very minor.

In [ ]:
# UFR CURVE 2015

ufr_2015_crv = ql.UltimateForwardTermStructure(
        ftk_handle, 
        llfr_2015_handle, 
        ufr_2015_handle, 
        FIRST_SMOOTHING_POINT_2015, 
        ALPHA_2015)
ufr_2015_crv.enableExtrapolation()
ufr_2015_handle = ql.YieldTermStructureHandle(ufr_2015_crv)

In [ ]:
# UFR CURVE 2024

ufr_2024_crv = ql.UltimateForwardTermStructure(
        ftk_handle, 
        llfr_2024_handle, 
        ufr_2024_handle, 
        FIRST_SMOOTHING_POINT_2024, 
        ALPHA_2024)
ufr_2024_crv.enableExtrapolation()
ufr_2024_handle = ql.YieldTermStructureHandle(ufr_2024_crv)

Both versions of the UFR curve are constructed above.

In [ ]:
# UFR CURVE 2021

def weighting_function(first: float, second: float) -> float:
    return 0.75 * first + 0.25 * second

ufr_2021_crv = ql.CompositeZeroYieldStructure(ufr_2015_handle, ufr_2024_handle, weighting_function, ql.Compounded, ql.Annual)
ufr_2021_crv.enableExtrapolation()
ufr_2021_handle = ql.YieldTermStructureHandle(ufr_2021_crv)

The DNB decided to introduce the new UFR methodology (labelled as 2024) gradually. The transition begins in 2021 and assumes that between 2021-2023 a hybrid version of the UFR curve will be in use to smoothen the impact of moving to the new methodology.

This means that the zero, annually compounded rates from the UFR curve will be a weighted average of the zero rates (with the same compounding) computed by both UFR 2015 and UFR 2024 curves, i.e.

\begin{equation*}
z_{2021}(t,T) = \gamma_{2015} \times z_{2015}(t,T) + \gamma_{2024} \times z_{2024}(t,T),
\end{equation*}

where $t \leq T$, $\gamma_{2015} + \gamma_{2024} = 1$. In fact, for 2021 reporting we have $\gamma_{2015}=0.75$ and $\gamma_{2024}=0.25$.

`QuantLib` allows to construct such a term structure with `ql.CompositeZeroYieldStructure`. We need to pass the two term structures, a weighting function defining the relationship between the zero rates and the compounding which those rates should follow. That is done in the cell above. 

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

# READ IN THE LIABILITIES

liabilities = read_liabilities()

# PRODUCE A PLOT THEREOF

plt.figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
width = 100

title=f'Stylized set of liabilities'
plt.title(title)

dates, flows = zip(*liabilities)
dates = [datetime.datetime(d.year(), d.month(), d.dayOfMonth()) for d in dates]
plt.bar(dates, flows, width, color='#222f3e', label="Liabilities")
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}')) # No decimal places
plt.xlabel('maturity')
plt.ylabel('amount in EUR')
plt.show()

We use a stylized set of pension liabilities, summarized in the graph above. 

In [ ]:
def liabilities_pricer(discount_handle: ql.YieldTermStructureHandle):
    
    def value(cash_flows: List[Tuple[ql.Date, float]]) -> float:
        return sum([discount_handle.discount(dt) * amount for dt, amount in cash_flows])

    return value


ftk_pricer = liabilities_pricer(ftk_handle)
ufr_2015_pricer = liabilities_pricer(ufr_2015_handle)
ufr_2024_pricer = liabilities_pricer(ufr_2024_handle)
ufr_2021_pricer = liabilities_pricer(ufr_2021_handle)

There are many ways to compute a an `npv` of a set of cash flows in `QuantLib`. What we do is capture the discount handle and use a callable pricer to discount the liabilities. 

The obtained values are presented below.

In [ ]:
npv_ftk = ftk_pricer(liabilities)
print('FTK NPV ', '{:,.0f}'.format(npv_ftk))

npv_ufr_2015 = ufr_2015_pricer(liabilities)
print('UFR 2015 NPV ', '{:,.0f}'.format(npv_ufr_2015))

npv_ufr_2024 = ufr_2024_pricer(liabilities)
print('UFR 2024 NPV ', '{:,.0f}'.format(npv_ufr_2024))

npv_ufr_2021 = ufr_2021_pricer(liabilities)
print('UFR 2021 NPV ', '{:,.0f}'.format(npv_ufr_2021))

In [ ]:
def zero_rates(curve_handle: ql.YieldTermStructureHandle, tenors: List[ql.Period]) -> List[Tuple[float, float]]:
    reference_date = curve_handle.referenceDate()
    d_counter = curve_handle.dayCounter()
    dates = [reference_date + tnr for tnr in tenors]
    t = [curve_handle.timeFromReference(d) for d in dates]
    z = [curve_handle.zeroRate(d, d_counter, ql.Compounded, ql.Annual).rate() for d in dates]
    return list(zip(t, z))


tenors = [ql.PeriodParser.parse(str(i) + 'y') for i in range(1, 101)]

plt.figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
plt.title(f'Comparison of zero rates for the market, UFR 2015, 2024 and 2021 curves as of {today}')
plt.xlabel('years to maturity')
plt.ylabel('zero rate')

t, ftk_zeroes = zip(*zero_rates(ftk_handle, tenors))
plt.plot(t, ftk_zeroes, "#222f3e", label="Market / FTK")

_, ufr_2015_zeroes = zip(*zero_rates(ufr_2015_handle, tenors))
plt.plot(t, ufr_2015_zeroes, "#00FF7F", label="UFR 2015")

_, ufr_2021_zeroes = zip(*zero_rates(ufr_2021_handle, tenors))
plt.plot(t, ufr_2021_zeroes, "#00d2d3", label="UFR 2021")

_, ufr_2024_zeroes = zip(*zero_rates(ufr_2024_handle, tenors))
plt.plot(t, ufr_2024_zeroes, "#F96714", label="UFR 2024")

plt.legend(loc="upper left")
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2%}'))
plt.show()

The graph shows the differences between the zero rates (annually compounded) for different flavours of the UFR curve, as well as the market curve.

The term structures behave in line with the intuition. UFR 2015 starts to deviate from the market / FTK curve past 20 years tenor, which is its first smoothing point threshold and increases to the level of roughly 1.25% at 100 years to maturity. 

UFR 2024 is following the market levels up to 30 years tenor, in line with its first smoothing point. The curve is noticeable flatter than the 2015 counterpart. 

Finally, in between the two we have the weighted curve.


In [ ]:
dnb_rates = [r for _, r in read_dnb_ufr_zero_rates(today)]
differences = [(z_dnb-z_ql) * 10000 for z_dnb, z_ql in zip(dnb_rates, ufr_2021_zeroes)]

plt.figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
plt.title(f'Differences between the expected (DNB) and the computed compounded UFR zero rates')
plt.xlabel('years to maturity')
plt.ylabel('difference in bps')

plt.plot(differences, color="#222f3e", label="difference in bps")
plt.plot([0.05 for i in range(100)], linestyle='--', color='#F96714', label="difference bandwith")
plt.plot([-0.05 for i in range(100)], linestyle='--', color='#F96714') 
plt.ylim(-3.2, 1.1)
plt.legend(loc="upper right")
plt.show()

We can also compare the computed annually compounded zero rates with the official rates published by the DNB. 

Given that the required precision is up to the 5th decimal point, the computed rates should not deviate from the official ones by more than 0.05 bps. The plot above illustrates the acceptable bandwith.

After rounding the computed rates as well, the expected total difference should be equal to zero. **Although, given that the market data set comes from a different data source, we do expect to observe differences, as shown above.** Those differences would disappear completely if the official data was used and the graph would look more like the one below (note the change in scale on the y-axis).

In [ ]:
from IPython.display import Image
Image(filename='data/differences.png') 

In [ ]:
def bump_quote(quote_handle: ql.RelinkableQuoteHandle, bump = 0.0001):
    quote_handle.linkTo(ql.SimpleQuote(quote_handle.value() + bump))
    

BUMP = 1.0e-4
SCALING = 1.0e-4

ftk_sensitivities = []
ufr_2015_sensitivities = []
ufr_2024_sensitivities = []
ufr_2021_sensitivities = []

for tenor, quote in swap_quotes:  
    bump_quote(quote, BUMP)
    ftk_sensitivities.append((tenor , (ftk_pricer(liabilities) - npv_ftk) / BUMP * SCALING))
    ufr_2015_sensitivities.append((tenor , (ufr_2015_pricer(liabilities) - npv_ufr_2015) / BUMP * SCALING))    
    ufr_2021_sensitivities.append((tenor , (ufr_2021_pricer(liabilities) - npv_ufr_2021) / BUMP * SCALING))
    ufr_2024_sensitivities.append((tenor , (ufr_2024_pricer(liabilities) - npv_ufr_2024) / BUMP * SCALING))
    bump_quote(quote, -BUMP)

The final step is to provide some insight into the sensitivity of the present value of the liabilities wrt the market quotes. 

The sensitivity is computed numerically. At this point, the earlier setup with the observer pattern shows its added value. Every time `bump_quote` is called on one of the market quotes, the observer receives a notification and the value of the LLFR handle is updated, next to the market / FTK handle which is also notified. 

In the end we obtain some interesting sensitivity distributions. 

While the market delta is centered around the 40 years bucket, the UFR curves' sensitivities concentrate around 25 and 30 years' buckets, leaving very little sensitivity for 40 and 50 years' buckets. 

In [ ]:
import numpy

plt.figure(num=None, figsize=(10, 5), dpi=100, facecolor='w', edgecolor='k')
width = 0.2
x_axis = numpy.arange(len(swap_quotes))

title=f'Par deltas as of {today}'
plt.title(title)
plt.xlabel('tenor')
plt.ylabel('par deltas')

grid, ftk_deltas = zip(*ftk_sensitivities)
grid_str = [str(tnr) for tnr in grid]
plt.bar(x_axis, ftk_deltas, width, color='#222f3e', label="Market \ FTK")

_, ufr_2015_deltas = zip(*ufr_2015_sensitivities)
plt.bar(x_axis + width, ufr_2015_deltas, width, alpha=0.8, color='#00FF7F', label="UFR 2015")

_, ufr_2021_deltas = zip(*ufr_2021_sensitivities)
plt.bar(x_axis + 3 * width, ufr_2021_deltas, width, alpha=0.8, color='#00d2d3', label="UFR 2021")

_, ufr_2024_deltas = zip(*ufr_2024_sensitivities)
plt.bar(x_axis + 2 * width, ufr_2024_deltas, width, alpha=0.8, color='#F96714', label="UFR 2024")

plt.legend(loc="lower left")
plt.xticks(x_axis, grid_str)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
plt.show()